json.load() - read JSON file & convert into a python dictionary/list


json.loads() - convert JSON String to a dictionary



json.dump() - writes JSON directly to a JSON file like OBJECT 


json.dumps() - returns JSON string representation of the python DICT{'age':59} to {"age":59} 


In [14]:
import pprint
import requests 
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
import csv


In [15]:
import datetime

now = datetime.datetime(2025,7,1)
print(now)

2025-07-01 00:00:00


In [16]:
url = "https://api.open-meteo.com/v1/forecast?latitude=51.5072&longitude=0.1276&current=temperature_2m,relative_humidity_2m,wind_speed_10m,rain,precipitation,weather_code&timezone=Europe%2FLondon"
response = requests.get(url)
data = response.json()
pprint.pp(data)

from datetime import datetime
summary = {
    "date":datetime.now().strftime("%Y-%m-%d"),
    "time": data["current"]["time"],
    "temperature":data["current"]["temperature_2m"],
    "relative_humidity": data["current"]["relative_humidity_2m"],
    "wind_speed": data["current"]["wind_speed_10m"],
    "rain": data["current"]["rain"],
    "precipitation": data["current"]["precipitation"],
    "weather_code": data["current"]["weather_code"]
}
df = pd.DataFrame([summary])

print(df)

{'latitude': 51.5,
 'longitude': 0.119999886,
 'generationtime_ms': 0.07724761962890625,
 'utc_offset_seconds': 3600,
 'timezone': 'Europe/London',
 'timezone_abbreviation': 'GMT+1',
 'elevation': 3.0,
 'current_units': {'time': 'iso8601',
                   'interval': 'seconds',
                   'temperature_2m': '°C',
                   'relative_humidity_2m': '%',
                   'wind_speed_10m': 'km/h',
                   'rain': 'mm',
                   'precipitation': 'mm',
                   'weather_code': 'wmo code'},
 'current': {'time': '2025-07-21T12:45',
             'interval': 900,
             'temperature_2m': 18.6,
             'relative_humidity_2m': 73,
             'wind_speed_10m': 6.2,
             'rain': 3.4,
             'precipitation': 3.4,
             'weather_code': 2}}
         date              time  temperature  relative_humidity  wind_speed  \
0  2025-07-21  2025-07-21T12:45         18.6                 73         6.2   

   rain  precipitatio

In [17]:
# export as csv
filename = f"/Users/madalinasamoila/airflow-weather-project/hourly_summary_{summary['date']}.csv"
df.to_csv(filename, index= False)

In [18]:
input_file = f"/Users/madalinasamoila/airflow-weather-project/hourly_summary_2025-07-21.csv"
output_file =  f"/Users/madalinasamoila/airflow-weather-project/daily_summary_2025-07-21.csv"
df = pd.read_csv(input_file)
df

,date,time,temperature,relative_humidity,wind_speed,rain,precipitation,weather_code
0,2025-07-21,2025-07-21T12:45,18.6,73,6.2,3.4,3.4,2


In [19]:
# Create new dataframe to store daily_summary data
summary = {
    "date": df["date"].iloc[0],
    "min_temperature": df["temperature"].min(),
    "max_temperature": df["temperature"].min(),
    "avg_wind_speed": round(df["wind_speed"].mean(), 2),
    "rain_expected": df["rain"].sum() > 0
}

df_summary = pd.DataFrame([summary])

df_summary.to_csv(output_file, index= False)
print(f"Saved daily summary to {output_file}")

Saved daily summary to /Users/madalinasamoila/airflow-weather-project/daily_summary_2025-07-21.csv


In [20]:
from datetime import datetime, timedelta
import pandas as pd
import os

In [21]:
yesterday = (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d")
today = (datetime.now()).strftime("%Y-%m-%d")
today

'2025-07-21'

In [22]:
input_file = f"/Users/madalinasamoila/airflow-weather-project/hourly_summary_2025-07-21.csv"
output_file =  f"/Users/madalinasamoila/airflow-weather-project/daily_summary_2025-07-21.csv"
df = pd.read_csv(input_file)
df

,date,time,temperature,relative_humidity,wind_speed,rain,precipitation,weather_code
0,2025-07-21,2025-07-21T12:45,18.6,73,6.2,3.4,3.4,2


In [23]:

summary_file = f"/Users/madalinasamoila/airflow-weather-project/daily_summary_2025-07-21.csv"
hourly_file = f"/Users/madalinasamoila/airflow-weather-project/hourly_summary_2025-07-21.csv"
df_summary = pd.read_csv(summary_file)
df_hourly = pd.read_csv(hourly_file)

In [24]:
df_hourly

,date,time,temperature,relative_humidity,wind_speed,rain,precipitation,weather_code
0,2025-07-21,2025-07-21T12:45,18.6,73,6.2,3.4,3.4,2


In [25]:
rain_codes = {51, 53, 55, 56, 57, 61, 63, 65, 66, 67, 80, 81, 82}
snow_codes = {71, 73, 75, 77, 85, 86}
thunderstorm_codes = {95, 96, 99}
snow_expected = any(df_hourly["weather_code"].isin(snow_codes))

temp = df_summary.loc[0,"max_temperature"]
rain_expected = df_summary.loc[0,"rain_expected"]
snow_expected = 'False'

alert_messages = []

weather_codes_today = set(df_hourly["weather_code"].dropna().astype(int))

if weather_codes_today & rain_codes:
    alert_messages.append(f"🌧️ Rain expected today. Weather code is {weather_codes_today}. Take precautions.")

# Snow alert
if weather_codes_today & snow_codes:
    alert_messages.append("❄️ Snow expected today. Weather code is {weather_codes_today}. Schedule maintenance.")

# Thunderstorm alert
if weather_codes_today & thunderstorm_codes:
    alert_messages.append("⛈️ Thunderstorm alert! Weather code is {weather_codes_today}. Stay safe and prepare response teams.")

if alert_messages:
    alert_text = "\n".join(alert_messages)
    print(alert_text)
        # Optional: send_email(to="your_email@example.com", subject="Extreme Weather Alert", html_content=alert_text)
else:
    print("✅ No extreme weather conditions detected today.")

    


✅ No extreme weather conditions detected today.


In [28]:
import mysql.connector
def load_df(df, table_name):
    try:
        # Get database credentials from .env file
        db_host = "localhost"
        db_user = "root"
        db_password = ''
        db_database = "apache_weather_data"

        # Create a connection to the database
        conn = mysql.connector.connect(
            host=db_host, user=db_user, password=db_password, database=db_database
        )

        # Create a SQLAlchemy engine
        engine = create_engine(
            f"mysql+mysqlconnector://{db_user}:{db_password}@{db_host}/{db_database}"
        )

        # Insert DataFrame into a SQL table, with a name of table_name given
        df.to_sql(name=table_name, con=engine, if_exists="replace", index=False)

        # Close the database connection
        conn.close()

    except Exception as error:
        print(f"An error occurred: {error}")

load_df(df_hourly, 'weather_hourly')
load_df(df_summary, 'weather_daily_summary')
